My project to assess the demographics of Lynn MA for Family & Childrens Services of Greater Lynn is going too slow to make the course deadline for report submittal.  I've hit too many problems relating to the relationship between this MacBook Pro and the Jupyter Notebook and with establishing and environment in which geopandas can be reliably imported.  I'll continue the project because of the high value for me, however, for report submittal, I'll do something that does not require using shape files.

This study compares several communities on the Massachusetts Northshore for settling.

In [303]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import lxml
import html5lib
from bs4 import BeautifulSoup
import requests
import csv

The following cells download Zip Codes for the Northshore communities.  One town has several zip codes, other towns have one zip code. For the latter set of zip codes, the Neighborhood is made to be the same as the town.

In [306]:
z_file='https://github.com/ItsMeLarry/Coursera_Capstone/raw/master/NorthshoreZipCodes.csv'
resp = requests.get(z_file)
output = open('zcodes.csv', 'wb')
output.write(resp.content)
nszcdf=pd.read_csv('zcodes.csv',index_col=False)
nszcdf.head()
print(nszcdf.shape)

(40, 3)


In [307]:
nsdf=pd.DataFrame(columns=['Zip Code','Town','Neighborhood'],dtype=str)
for i in range(0,nszcdf.shape[0]):
    if pd.isnull(nszcdf.loc[i,'Neighborhood']):
        nszcdf.loc[i,'Neighborhood']=nszcdf.loc[i,'Town'];
nszcdf.head(10)

,Zip Code,Town,Neighborhood
0,1901,Lynn,Central Lynn
1,1902,Lynn,East Lynn
2,1904,Lynn,Wyoma
3,1905,Lynn,West Lynn
4,1906,Saugus,North Saugus
5,1906,Saugus,Merrimac Terrace
6,1906,Saugus,Oakland Vale
7,1906,Saugus,Lynnhurst
8,1906,Saugus,Saugus
9,1906,Saugus,Pleasant Hills


Now, to eliminate Zip Code duplication by combining neighborhoods in one Zip Code row. A dataframe is created using Groupby, grouping on Zip Code and Town together. Commas are inserted into the Neighborhood entries via an .apply to the join of the data. A reset_index is used to make the index of the original groupby become the first two columns, i.e. Zip code and Town. The original Groupby sorted the Zip Codes.

In [308]:
nszcdf2=pd.DataFrame(nszcdf.groupby(['Zip Code','Town'])['Neighborhood'].apply(', '.join),columns=['Neighborhood']).reset_index()
nszcdf2.rename(columns={'Zip Code':'Zip_Code'},inplace=True)
nszcdf2.head(10)


,Zip_Code,Town,Neighborhood
0,1901,Lynn,Central Lynn
1,1902,Lynn,East Lynn
2,1904,Lynn,Wyoma
3,1905,Lynn,West Lynn
4,1906,Saugus,"North Saugus, Merrimac Terrace, Oakland Vale, ..."
5,1907,Swampscott,Swampscott
6,1908,Nahant,"Nahant, Little Nahant, Pea Island, Castle Rock"
7,1915,Beverly,"Rial Side, North Beverly, Beverly Cove, Prides..."
8,1923,Danvers,"Metro, Putnamville, Tapleyville, Danversport, ..."
9,1937,Danvers,Hathorne


In [309]:
nszcdf2.shape

(13, 3)

13 Zip Codes, each with the Town and the list of Neighborhoods.

Now, to add the longitude and latitude to the zip codes.

In [310]:
zcll='http://federalgovernmentzipcodes.us/free-zipcode-database-Primary.csv'
resp = requests.get(zcll)
output = open('zcllf.csv', 'wb')
output.write(resp.content)
zclldf=pd.read_csv('zcllf.csv',index_col=False);zclldf.set_index('Zipcode',inplace=True)
zclldf.head()

,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
Zipcode,,,,,,,,,,,
705,STANDARD,AIBONITO,PR,PRIMARY,18.14,-66.26,NA-US-PR-AIBONITO,False,NaN,NaN,NaN
610,STANDARD,ANASCO,PR,PRIMARY,18.28,-67.14,NA-US-PR-ANASCO,False,NaN,NaN,NaN
611,PO BOX,ANGELES,PR,PRIMARY,18.28,-66.79,NA-US-PR-ANGELES,False,NaN,NaN,NaN
612,STANDARD,ARECIBO,PR,PRIMARY,18.45,-66.73,NA-US-PR-ARECIBO,False,NaN,NaN,NaN
601,STANDARD,ADJUNTAS,PR,PRIMARY,18.16,-66.72,NA-US-PR-ADJUNTAS,False,NaN,NaN,NaN


In [311]:
for i in range(0,nszcdf2.shape[0]):
    nszcdf2.loc[i,'Lat']=zclldf.loc[nszcdf2.loc[i,'Zip_Code'],'Lat']
    nszcdf2.loc[i,'Long']=zclldf.loc[nszcdf2.loc[i,'Zip_Code'],'Long']
nszcdf2.head()

,Zip_Code,Town,Neighborhood,Lat,Long
0,1901,Lynn,Central Lynn,42.47,-70.96
1,1902,Lynn,East Lynn,42.47,-70.96
2,1904,Lynn,Wyoma,42.47,-70.96
3,1905,Lynn,West Lynn,42.47,-70.96
4,1906,Saugus,"North Saugus, Merrimac Terrace, Oakland Vale, ...",42.46,-71.01


The different Lynn zip codes all have the same lat and long.  That needs to be fixed.  First, a map is necessary to assure no other location problems also exist.

In [312]:
nszcdf2.sort_values("Zip_Code",inplace=True)
nszcdf2.head()

,Zip_Code,Town,Neighborhood,Lat,Long
0,1901,Lynn,Central Lynn,42.47,-70.96
1,1902,Lynn,East Lynn,42.47,-70.96
2,1904,Lynn,Wyoma,42.47,-70.96
3,1905,Lynn,West Lynn,42.47,-70.96
4,1906,Saugus,"North Saugus, Merrimac Terrace, Oakland Vale, ...",42.46,-71.01


Bring in the libraries needed to cluster and map the town and neighnorhood data.

In [313]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare 
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


Make an overview map that shows all the zip codes in the boroughs.  Use Nominatim, folium and geolocator.

In [314]:

address = 'Lynn, MA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Lynn are {}, {}.'.format(latitude, longitude))
    # create map of the Northshore using latitude and longitude values
map_Northshore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Town, Zip_Code in zip(nszcdf2['Lat'], nszcdf2['Long'],nszcdf2['Town'], nszcdf2['Zip_Code']):
    label = '{}, {}'.format(Zip_Code, Town)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Northshore)  
map_Northshore

The geographical coordinate of Lynn are 42.466763, -70.9494939.


Time to fix the Lynn latitude/longitude problem. Danvers also has the same Lat and Long for two zip codes. Use the map to find addresses or landmarks within the zip code areas.

In [315]:
address = 'North Shore Community College Lynn Campus, Lynn, MA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
lat1901 = location.latitude
long1901 = location.longitude
print('The geographical coordinate of {} are {}, {}.'.format(address, lat1901, long1901))

address = 'Union Hospital, Lynn, MA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
lat1904 = location.latitude
long1904 = location.longitude
print('The geographical coordinate of {} are {}, {}.'.format(address, lat1904, long1904))

address = 'Essex Street, Lynn, MA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
lat1902 = location.latitude
long1902 = location.longitude
print('The geographical coordinate of {} are {}, {}.'.format(address, lat1902, long1902))

address = 'Barry Park, Lynn, MA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
lat1905 = location.latitude
long1905 = location.longitude
print('The geographical coordinate of {} are {}, {}.'.format(address, lat1906, long1906))

address = 'Hathorne, Danvers, MA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
lat1937 = location.latitude
long1937 = location.longitude
print('The geographical coordinate of {} are {}, {}.'.format(address, lat1937, long1937))

#address = 'Barry Park, Lynn, MA'
#geolocator = Nominatim(user_agent="to_explorer")
#location = geolocator.geocode(address)
#lat1923 = location.latitude
#long1923 = location.longitude
#print('The geographical coordinate of {} are {}, {}.'.format(address, lat1923, long1923))

The geographical coordinate of North Shore Community College Lynn Campus, Lynn, MA are 42.46028995, -70.9429028440395.
The geographical coordinate of Union Hospital, Lynn, MA are 42.5010719, -70.9796327771149.
The geographical coordinate of Essex Street, Lynn, MA are 42.470135, -70.938967.
The geographical coordinate of Barry Park, Lynn, MA are 42.46207585, -70.9746762509888.
The geographical coordinate of Hathorne, Danvers, MA are 42.5862053, -70.9744964.


Update the Northshore dataframe with the improved zip codes.

In [316]:
#nszcdf2['Zip_Code']=nszcdf2.Zip_Code.astype(int)
nszcdf2['Lat']=np.where(nszcdf2['Zip_Code']==1901,lat1901,nszcdf2['Lat'])
nszcdf2['Long']=np.where(nszcdf2['Zip_Code']==1901, long1901,nszcdf2['Long'])
nszcdf2['Lat']=np.where(nszcdf2['Zip_Code']==1902,lat1902,nszcdf2['Lat'])
nszcdf2['Long']=np.where(nszcdf2['Zip_Code']==1902, long1902,nszcdf2['Long'])
nszcdf2['Lat']=np.where(nszcdf2['Zip_Code']==1904,lat1904,nszcdf2['Lat'])
nszcdf2['Long']=np.where(nszcdf2['Zip_Code']==1904, long1904,nszcdf2['Long'])
nszcdf2['Lat']=np.where(nszcdf2['Zip_Code']==1937,lat1937,nszcdf2['Lat'])
nszcdf2['Long']=np.where(nszcdf2['Zip_Code']==1937, long1937,nszcdf2['Long'])
nszcdf2.head(30)

,Zip_Code,Town,Neighborhood,Lat,Long
0,1901,Lynn,Central Lynn,42.460290,-70.942903
1,1902,Lynn,East Lynn,42.470135,-70.938967
2,1904,Lynn,Wyoma,42.501072,-70.979633
3,1905,Lynn,West Lynn,42.470000,-70.960000
4,1906,Saugus,"North Saugus, Merrimac Terrace, Oakland Vale, ...",42.460000,-71.010000
5,1907,Swampscott,Swampscott,42.460000,-70.890000
6,1908,Nahant,"Nahant, Little Nahant, Pea Island, Castle Rock",42.420000,-70.920000
7,1915,Beverly,"Rial Side, North Beverly, Beverly Cove, Prides...",42.550000,-70.840000
8,1923,Danvers,"Metro, Putnamville, Tapleyville, Danversport, ...",42.570000,-70.950000
9,1937,Danvers,Hathorne,42.586205,-70.974496


In [317]:
map_Northshore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Town, Zip_Code in zip(nszcdf2['Lat'], nszcdf2['Long'],nszcdf2['Town'], nszcdf2['Zip_Code']):
    label = '{}, {}'.format(Zip_Code, Town)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Northshore)  
map_Northshore

Lynn and Danvers zip code problem fixed.

Lynn is the largest town, with 4 zip codes with neighborhood collections.  A close look at Lynn to be done.

In [318]:
lynndf=nszcdf2[nszcdf2['Town']=='Lynn'].reset_index(drop=True)
print(lynndf.head())
address = 'Lynn, MA'

geolocator = Nominatim(user_agent="sc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lynn, MA are {}, {}.'.format(latitude, longitude))
print('Lynn has', lynndf.shape[0],'Zip Codes.')

   Zip_Code  Town  Neighborhood        Lat       Long
0      1901  Lynn  Central Lynn  42.460290 -70.942903
1      1902  Lynn     East Lynn  42.470135 -70.938967
2      1904  Lynn         Wyoma  42.501072 -70.979633
3      1905  Lynn     West Lynn  42.470000 -70.960000
The geograpical coordinate of Lynn, MA are 42.466763, -70.9494939.
Lynn has 4 Zip Codes.


In [319]:
map_lynn = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, Zip_Code, Neighborhood in zip(lynndf['Lat'], lynndf['Long'], lynndf['Zip_Code'], lynndf['Neighborhood']):
    label = '{}, {}'.format(Zip_Code,Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lynn)  
map_lynn

Explore Lynn for venues.  Start by establishingh access info for using Foursquare.

In [320]:
CLIENT_ID = 'VCB1F2GAXJNP0ECNYPZ2MILKJ0EY0N4VQPKAHEBNYW4YSQI4' # your Foursquare ID
CLIENT_SECRET = 'MMDYZKGTQCPHKK0JYZGBHFSRDZ5ZNLB5TUNX4MVGITNBYNNR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


I'll choose the Wyoma neighborhood at first because, it is seperated from the others in Lynn.  One of the others can then be chosen to represent the group of them.

In [321]:
dahood=2
neighborhood_latitude = lynndf.loc[dahood, 'Lat'] # nteighborhood latitude value
neighborhood_longitude = lynndf.loc[dahood, 'Long'] # neighborhood longitude value

neighborhood_name = lynndf.loc[dahood, 'Neighborhood'] # neighborhood name
zipcode = lynndf.loc[dahood,'Zip_Code']
print('Latitude and longitude values of {} are {}, {}. The Zip Code is {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude,
                                                               zipcode))
radius=200
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
import requests
results = requests.get(url).json()
print('There are {} venues around the neighborhood of {}.'.format(len(results['response']['groups'][0]['items']),neighborhood_name))


Latitude and longitude values of Wyoma are 42.5010719, -70.9796327771149. The Zip Code is 1904.
There are 3 venues around the neighborhood of Wyoma.


Get the json and start working on it. Extract the venues for all the neighborhoods in Wyoma.

In [322]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7c6edddd57977bd6bd2bbc'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4f8b3789e4b0abaa04560e2c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/garden_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d15a941735',
         'name': 'Garden',
         'pluralName': 'Gardens',
         'primary': True,
         'shortName': 'Garden'}],
       'id': '4f8b3789e4b0abaa04560e2c',
       'location': {'address': 'Essex St',
        'cc': 'US',
        'city': 'Lynn',
        'country': 'United States',
        'distance': 138,
        'formattedAddress': ['Essex St', 'Lynn, MA 01901', 'United States'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 42.46570752124003,
          'lng': -70.950381

In [323]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head())
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

                 name               categories        lat        lng
0  SML Backyard/Patio                   Garden  42.465708 -70.950381
1            Richdale        Convenience Store  42.465973 -70.950219
2    USA Vein Clinics  Health & Beauty Service  42.466755 -70.949460
3 venues were returned by Foursquare.


Routine to get nearby venues for a neighborhood.

In [324]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Get the venues for all the Lynn neighborhoods. Create a dataframe with a line for each venue containing the latitude and longitude of the venue in the neighborhood.

In [325]:
rad=500
LIMIT=100
Lynn_venues = getNearbyVenues(names=lynndf['Neighborhood'],
                                   latitudes=lynndf['Lat'],
                                   longitudes=lynndf['Long'],
                                   radius=rad
                                    )
print(Lynn_venues.shape)
Lynn_venues.head()

Central Lynn
East Lynn
Wyoma
West Lynn
(42, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Lynn,42.46029,-70.942903,Lynn Museum/LynnArts,42.462967,-70.944153,Museum
1,Central Lynn,42.46029,-70.942903,Capitol Diner,42.463256,-70.944278,Diner
2,Central Lynn,42.46029,-70.942903,Tacos Lupita,42.463650,-70.945770,Mexican Restaurant
3,Central Lynn,42.46029,-70.942903,The Blue Ox,42.463125,-70.947415,Burger Joint
4,Central Lynn,42.46029,-70.942903,Land of a Thousand Hills Coffee Co.,42.462971,-70.946668,Coffee Shop


In [326]:
Lynn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Lynn,21,21,21,21,21,21
East Lynn,9,9,9,9,9,9
West Lynn,10,10,10,10,10,10
Wyoma,2,2,2,2,2,2


So, Central Lynn is the busiest area, followed by West Lynn.  Wyoma looks principly like a suburban community. Distribution of venues among neighborhoods.

In [327]:
# one hot encoding
lynn_onehot = pd.get_dummies(Lynn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lynn_onehot['Neighborhood'] = Lynn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lynn_onehot.columns[-1]] + list(lynn_onehot.columns[:-1])
lynn_onehot = lynn_onehot[fixed_columns]

lynn_onehot.head()


,Neighborhood,ATM,American Restaurant,Art Gallery,Auto Garage,Automotive Shop,Bar,Big Box Store,Brazilian Restaurant,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Chinese Restaurant,Coffee Shop,Convenience Store,Deli / Bodega,Diner,Discount Store,Donut Shop,Fast Food Restaurant,Furniture / Home Store,Gym / Fitness Center,Harbor / Marina,Laundromat,Mexican Restaurant,Museum,Pharmacy,Pizza Place,Pub,Salon / Barbershop,Spanish Restaurant,State / Provincial Park,Storage Facility,Video Store
0,Central Lynn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Central Lynn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Central Lynn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Central Lynn,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Central Lynn,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Normalize the distribution of venues.

In [328]:
print(lynn_onehot.shape)
lynn_grouped = lynn_onehot.groupby('Neighborhood').mean().reset_index()
print(lynn_grouped.shape)
lynn_grouped


(42, 34)
(4, 34)


,Neighborhood,ATM,American Restaurant,Art Gallery,Auto Garage,Automotive Shop,Bar,Big Box Store,Brazilian Restaurant,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Chinese Restaurant,Coffee Shop,Convenience Store,Deli / Bodega,Diner,Discount Store,Donut Shop,Fast Food Restaurant,Furniture / Home Store,Gym / Fitness Center,Harbor / Marina,Laundromat,Mexican Restaurant,Museum,Pharmacy,Pizza Place,Pub,Salon / Barbershop,Spanish Restaurant,State / Provincial Park,Storage Facility,Video Store
0,Central Lynn,0.0,0.095238,0.047619,0.047619,0.047619,0.095238,0.0,0.000000,0.047619,0.047619,0.0,0.000000,0.095238,0.0,0.047619,0.047619,0.000000,0.047619,0.0,0.0,0.0,0.095238,0.000000,0.047619,0.047619,0.000000,0.000000,0.0,0.0,0.047619,0.047619,0.047619,0.000000
1,East Lynn,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.0,0.111111,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.111111,0.000000,0.000000,0.111111,0.111111,0.0,0.0,0.222222,0.000000,0.000000,0.111111
2,West Lynn,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.0,0.100000,0.000000,0.100000,0.100000,0.1,0.1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.1,0.000000,0.000000,0.000000,0.000000
3,Wyoma,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000


Identify the top 5 for each neighborhood.

In [329]:
num_top_venues = 5

for hood in lynn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = lynn_grouped[lynn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Lynn----
                 venue  freq
0      Harbor / Marina  0.10
1  American Restaurant  0.10
2                  Bar  0.10
3          Coffee Shop  0.10
4   Spanish Restaurant  0.05


----East Lynn----
                  venue  freq
0    Spanish Restaurant  0.22
1        Discount Store  0.11
2  Brazilian Restaurant  0.11
3           Pizza Place  0.11
4              Pharmacy  0.11


----West Lynn----
                       venue  freq
0                        ATM   0.1
1  Cajun / Creole Restaurant   0.1
2         Salon / Barbershop   0.1
3                        Pub   0.1
4     Furniture / Home Store   0.1


----Wyoma----
                    venue  freq
0    Gym / Fitness Center   0.5
1       Convenience Store   0.5
2                     ATM   0.0
3                  Museum   0.0
4  Furniture / Home Store   0.0




In [330]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

And the most common venues for each of the neighborhoods.

In [331]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lynn_grouped['Neighborhood']

for ind in np.arange(lynn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lynn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Lynn,Bar,American Restaurant,Harbor / Marina,Coffee Shop,Spanish Restaurant
1,East Lynn,Spanish Restaurant,Video Store,Pharmacy,Brazilian Restaurant,Chinese Restaurant
2,West Lynn,Discount Store,Furniture / Home Store,Big Box Store,Cajun / Creole Restaurant,Deli / Bodega
3,Wyoma,Convenience Store,Gym / Fitness Center,Video Store,Breakfast Spot,Coffee Shop


This tells us a little about Lynn, but does not provide an adequately large set of data to cluster for an inference.
We'll do all this for the entire Northshore and see what we get.

In [332]:
rad=1500
LIMIT=100
ns_venues = getNearbyVenues(names=nszcdf2['Neighborhood'],
                                   latitudes=nszcdf2['Lat'],
                                   longitudes=nszcdf2['Long'],
                                   radius=rad
                                    )
print(ns_venues.shape)
ns_venues.tail()

Central Lynn
East Lynn
Wyoma
West Lynn
North Saugus, Merrimac Terrace, Oakland Vale, Lynnhurst, Saugus, Pleasant Hills, Cliftondale
Swampscott
Nahant, Little Nahant, Pea Island, Castle Rock
Rial Side, North Beverly, Beverly Cove, Prides Crossing, Centerville, Central Beverly
Metro, Putnamville, Tapleyville, Danversport, Ferncroft, Burleys Corner
Hathorne
Marblehead, Gerry Island, Clifton, Marblehead Neck
Peabody, West Peabody, South Peabody, Proctor
Salem Neck, South Salem, Jeggle Island
(558, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
553,"Salem Neck, South Salem, Jeggle Island",42.51,-70.9,Dunkin Donuts,42.522598,-70.895820,Donut Shop
554,"Salem Neck, South Salem, Jeggle Island",42.51,-70.9,CVS pharmacy,42.501014,-70.896064,Pharmacy
555,"Salem Neck, South Salem, Jeggle Island",42.51,-70.9,Walgreens,42.518460,-70.909237,Pharmacy
556,"Salem Neck, South Salem, Jeggle Island",42.51,-70.9,Bella Pizza,42.511461,-70.906369,Pizza Place
557,"Salem Neck, South Salem, Jeggle Island",42.51,-70.9,Healthworks Fitness Center for Women,42.512228,-70.908854,Gym


In [333]:
ns_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Lynn,58,58,58,58,58,58
East Lynn,57,57,57,57,57,57
Hathorne,20,20,20,20,20,20
"Marblehead, Gerry Island, Clifton, Marblehead Neck",65,65,65,65,65,65
"Metro, Putnamville, Tapleyville, Danversport, Ferncroft, Burleys Corner",33,33,33,33,33,33
"Nahant, Little Nahant, Pea Island, Castle Rock",16,16,16,16,16,16
"North Saugus, Merrimac Terrace, Oakland Vale, Lynnhurst, Saugus, Pleasant Hills, Cliftondale",100,100,100,100,100,100
"Peabody, West Peabody, South Peabody, Proctor",16,16,16,16,16,16
"Rial Side, North Beverly, Beverly Cove, Prides Crossing, Centerville, Central Beverly",8,8,8,8,8,8


In [334]:
# one hot encoding
ns_onehot = pd.get_dummies(ns_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ns_onehot['Neighborhood'] = ns_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ns_onehot.columns[-1]] + list(ns_onehot.columns[:-1])
ns_onehot = ns_onehot[fixed_columns]

ns_onehot.tail()


,Neighborhood,ATM,Accessories Store,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Big Box Store,Bike Trail,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Church,Clothing Store,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Service,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Football Stadium,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hockey Rink,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kids Store,Korean Restaurant,Lake,Laundromat,Lingerie Store,Liquor Store,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Post Office,Pub,Record Shop,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wings Joint,Women's Store,Yoga Studio
553,"Salem Neck, South Salem, Jeggle Island",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
554,"Salem Neck, South Salem, Jeggle Island",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
555,"Salem Neck, South Salem, Jeggle Island",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
556,"Salem Neck, South Salem, Jeggle Island",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
557,"Salem Neck, South Salem, Jeggle Island",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [335]:
print(ns_onehot.shape)
ns_grouped = ns_onehot.groupby('Neighborhood').mean().reset_index()
print(ns_grouped.shape)
print(ns_grouped)

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ns_grouped['Neighborhood']

for ind in np.arange(ns_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ns_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

(558, 151)
(13, 151)
                                         Neighborhood       ATM  \
0                                        Central Lynn  0.017241   
1                                           East Lynn  0.000000   
2                                            Hathorne  0.000000   
3   Marblehead, Gerry Island, Clifton, Marblehead ...  0.000000   
4   Metro, Putnamville, Tapleyville, Danversport, ...  0.000000   
5      Nahant, Little Nahant, Pea Island, Castle Rock  0.000000   
6   North Saugus, Merrimac Terrace, Oakland Vale, ...  0.000000   
7       Peabody, West Peabody, South Peabody, Proctor  0.000000   
8   Rial Side, North Beverly, Beverly Cove, Prides...  0.000000   
9              Salem Neck, South Salem, Jeggle Island  0.000000   
10                                         Swampscott  0.000000   
11                                          West Lynn  0.013889   
12                                              Wyoma  0.000000   

    Accessories Store  American Restaura

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Lynn,Donut Shop,Bowling Alley,Beach,Pizza Place,Pharmacy
1,East Lynn,Pizza Place,Italian Restaurant,Pharmacy,Sandwich Place,American Restaurant
2,Hathorne,Hotel,Donut Shop,Italian Restaurant,Bank,Steakhouse
3,"Marblehead, Gerry Island, Clifton, Marblehead ...",Park,Seafood Restaurant,Beach,Yoga Studio,Bar
4,"Metro, Putnamville, Tapleyville, Danversport, ...",Pizza Place,Park,Italian Restaurant,American Restaurant,Chinese Restaurant


In [336]:
# set number of clusters
kclusters = 5

ns_grouped_clustering = ns_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ns_grouped_clustering)
ns_merged=[]
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 4, 0, 0, 3, 0, 2, 0, 1], dtype=int32)

In [337]:
# add clustering labels
print(kmeans.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ns_merged = nszcdf2
print(nszcdf2.shape, neighborhoods_venues_sorted.shape)
# merge Northshore_grouped with Northshore_data to add latitude/longitude for each neighborhood
ns_merged = ns_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#scarborough_merged[['Cluster Labels']]=scarborough_merged[['Cluster Labels']].astype('int32')
ns_merged[['Cluster Labels']]=ns_merged[['Cluster Labels']].astype('int32')
ns_merged.head(20) # check the last columns!


[0 0 0 0 0 4 0 0 3 0 2 0 1]
(13, 5) (13, 7)


,Zip_Code,Town,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1901,Lynn,Central Lynn,42.460290,-70.942903,0,Donut Shop,Bowling Alley,Beach,Pizza Place,Pharmacy
1,1902,Lynn,East Lynn,42.470135,-70.938967,0,Pizza Place,Italian Restaurant,Pharmacy,Sandwich Place,American Restaurant
2,1904,Lynn,Wyoma,42.501072,-70.979633,1,Trail,Home Service,American Restaurant,Convenience Store,Lake
3,1905,Lynn,West Lynn,42.470000,-70.960000,0,Donut Shop,Pizza Place,Asian Restaurant,Pharmacy,Liquor Store
4,1906,Saugus,"North Saugus, Merrimac Terrace, Oakland Vale, ...",42.460000,-71.010000,0,Clothing Store,Shoe Store,Sandwich Place,Donut Shop,Women's Store
5,1907,Swampscott,Swampscott,42.460000,-70.890000,2,Beach,Tailor Shop,Playground,Pizza Place,Diner
6,1908,Nahant,"Nahant, Little Nahant, Pea Island, Castle Rock",42.420000,-70.920000,4,Park,Beach,Pizza Place,Harbor / Marina,Event Space
7,1915,Beverly,"Rial Side, North Beverly, Beverly Cove, Prides...",42.550000,-70.840000,3,Beach,Scenic Lookout,Hotel,Health & Beauty Service,Intersection
8,1923,Danvers,"Metro, Putnamville, Tapleyville, Danversport, ...",42.570000,-70.950000,0,Pizza Place,Park,Italian Restaurant,American Restaurant,Chinese Restaurant
9,1937,Danvers,Hathorne,42.586205,-70.974496,0,Hotel,Donut Shop,Italian Restaurant,Bank,Steakhouse


In [338]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ns_merged['Lat'], ns_merged['Long'], ns_merged['Neighborhood'], ns_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Because some neighborhoods of the Northshore are remote, a larger radius (1,500 m) for Foursquare was necessary for finding venues for ALL the neighborhoods.

The radius problem is driven by Cluster 1, the area of Wyoma in Lynn, a residential area distant from Lynn's commercial heart and bordering an expressway and a main four lane highway.

Cluster 0 is a group of exurban neighborhoods combining commercial zones with developments like shopping malls and marinas with suburban neighborhoods.  This cluster is characterized by many fast food eating venues.

Clusters 2, 3, 4 are the more geographically isolated coastal communities. They do not have many venues within them and they are separated by geography from the Cluster 0 communities and venues.  Nahant is an isolated penninsula and Beverly is similarly on a penninsula and across a river from the other communities.  Swampscott is small and in a "corner" of the area.

In [339]:
ns_merged.loc[ns_merged['Cluster Labels'] == 0, ns_merged.columns[[0]+[2] + list(range(5, ns_merged.shape[1]))]]

,Zip_Code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1901,Central Lynn,0,Donut Shop,Bowling Alley,Beach,Pizza Place,Pharmacy
1,1902,East Lynn,0,Pizza Place,Italian Restaurant,Pharmacy,Sandwich Place,American Restaurant
3,1905,West Lynn,0,Donut Shop,Pizza Place,Asian Restaurant,Pharmacy,Liquor Store
4,1906,"North Saugus, Merrimac Terrace, Oakland Vale, ...",0,Clothing Store,Shoe Store,Sandwich Place,Donut Shop,Women's Store
8,1923,"Metro, Putnamville, Tapleyville, Danversport, ...",0,Pizza Place,Park,Italian Restaurant,American Restaurant,Chinese Restaurant
9,1937,Hathorne,0,Hotel,Donut Shop,Italian Restaurant,Bank,Steakhouse
10,1945,"Marblehead, Gerry Island, Clifton, Marblehead ...",0,Park,Seafood Restaurant,Beach,Yoga Studio,Bar
11,1960,"Peabody, West Peabody, South Peabody, Proctor",0,Gym / Fitness Center,Paper / Office Supplies Store,Golf Course,Pizza Place,Liquor Store
12,1970,"Salem Neck, South Salem, Jeggle Island",0,Pizza Place,Coffee Shop,Donut Shop,Sandwich Place,Seafood Restaurant


In [340]:
ns_merged.loc[ns_merged['Cluster Labels'] == 1, ns_merged.columns[[0]+[2] + list(range(5, ns_merged.shape[1]))]]

,Zip_Code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,1904,Wyoma,1,Trail,Home Service,American Restaurant,Convenience Store,Lake


In [341]:
ns_merged.loc[ns_merged['Cluster Labels'] == 2, ns_merged.columns[[0]+[2] + list(range(5, ns_merged.shape[1]))]]

,Zip_Code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,1907,Swampscott,2,Beach,Tailor Shop,Playground,Pizza Place,Diner


In [342]:
ns_merged.loc[ns_merged['Cluster Labels'] == 3, ns_merged.columns[[0]+[2] + list(range(5, ns_merged.shape[1]))]]

,Zip_Code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,1915,"Rial Side, North Beverly, Beverly Cove, Prides...",3,Beach,Scenic Lookout,Hotel,Health & Beauty Service,Intersection


In [343]:
ns_merged.loc[ns_merged['Cluster Labels'] == 4, ns_merged.columns[[0]+[2] + list(range(5, ns_merged.shape[1]))]]

,Zip_Code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,1908,"Nahant, Little Nahant, Pea Island, Castle Rock",4,Park,Beach,Pizza Place,Harbor / Marina,Event Space
